<a href="https://colab.research.google.com/github/Nekoiii/ML_Practices_colab/blob/main/Lora_test_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sd-scripts github: https://github.com/kohya-ss/sd-scripts

Reference article：
· https://tkstock.site/2023/05/19/googlecolab-stablediffusion-lora-finetuning/#i-7

· https://www.ipentec.com/document/software-stable-diffusion-lora-learn#section_12



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python --version

Python 3.10.12


In [4]:
!git clone https://github.com/kohya-ss/sd-scripts /content/drive/MyDrive/StableDifussion/sd-scripts

Cloning into '/content/drive/MyDrive/StableDifussion/sd-scripts'...
remote: Enumerating objects: 3070, done.
remote: Counting objects: 100% (1483/1483), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 3070 (delta 1367), reused 1368 (delta 1320), pack-reused 1587
Receiving objects: 100% (3070/3070), 3.45 MiB | 16.82 MiB/s, done.
Resolving deltas: 100% (2122/2122), done.


In [5]:
#According to the sd-scripts github documentation, we should install PyTorch before install requirements.txt 
#because the versions of it depends on your environment and requirements.txt does not contain requirements for PyTorch.

%cd /content/drive/MyDrive/StableDifussion/sd-scripts
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install  --upgrade -r requirements.txt
#!pip install -U --pre triton
!pip install xformers==0.0.16rc425
%cd /content

/content/drive/MyDrive/StableDifussion/sd-scripts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu117 which is incompatible.
torchdata 0.6.1 r

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 33.4 MB/s eta 0:00:00
/content


In [6]:
# Change some config (Some user reports [ValueError: fp16 mixed precision requires a GPU] is occurred in training )
!accelerate config default --mixed_precision fp16

2023-06-13 07:45:55.823076: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 07:45:56.024758: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-13 07:45:56.858839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-06-13 07:45:56.858966: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

In [14]:
# Copy training images folder
training_data_path='/content/training_data'

%cp -r /content/drive/MyDrive/datasets/imgs/resized_canbright_imgs_1_partial '{training_data_path}'

In [15]:
# Generate captions and tags for each image.
%cd /content/drive/MyDrive/StableDifussion/sd-scripts

!python finetune/make_captions.py --batch_size 8 '{training_data_path}'
!python finetune/tag_images_by_wd14_tagger.py --batch_size 4 '{training_data_path}'

/content/drive/MyDrive/StableDifussion/sd-scripts
2023-06-13 08:43:16.383163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 08:43:16.580687: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-13 08:43:17.454523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-06-13 08:43:17.454610: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

In [16]:
# Merges captions with metadata and outputs the result in meta_cap.json.
# Merges Data Description (DD) tags with metadata from the previous step and generates a new file meta_cap_dd.json.
# Cleans and preprocesses the captions and tags from the previous step and generates a cleaned version in meta_clean.json.

!python finetune/merge_captions_to_metadata.py '{training_data_path}' meta_cap.json
!python finetune/merge_dd_tags_to_metadata.py '{training_data_path}' --in_json meta_cap.json meta_cap_dd.json
!python finetune/clean_captions_and_tags.py meta_cap_dd.json meta_clean.json

2023-06-13 08:44:07.410475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 08:44:07.608010: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-13 08:44:08.462649: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-06-13 08:44:08.462751: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

In [17]:
!python finetune/prepare_buckets_latents.py \
'{training_data_path}' meta_clean.json meta_lat.json \
/content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt \
--batch_size 4 \
--max_resolution 512,512 \
--mixed_precision no

2023-06-13 08:44:20.603441: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 08:44:20.800835: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-13 08:44:21.616483: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2023-06-13 08:44:21.616610: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

In [13]:
!pwd

/content/sd-scripts


In [28]:
!accelerate launch  \
--num_cpu_threads_per_process 1 \
train_network.py \
--pretrained_model_name_or_path=/content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt \
--in_json modified_meta_lat.json \
--train_data_dir='{training_data_path}' \
--output_dir=output_models \
--shuffle_caption \
--train_batch_size=1 \
--learning_rate=1e-4 \
--max_train_steps=5000 \
--use_8bit_adam \
--xformers --gradient_checkpointing \
--mixed_precision=fp16 \
--save_every_n_epochs=10 \
--save_precision=fp16 \
--network_module=networks.lora\
--training_comment="cb_satoo" \
#--save_model_as= safetensors
#--save_model_as= pt 
#--output_name=satoo_with_descriptions \


2023-06-13 10:26:14.686541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 10:26:14.875144: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-13 10:26:15.645136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-06-13 10:26:15.645216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

In [31]:
model_path='/content/drive/MyDrive/StableDifussion/sd-scripts/output_models/last.safetensors'
save_path='/content/drive/MyDrive/my_models/canbright_satoo/'
new_file_name = 'satoo_without_descriptions_5000.safetensors'

!mkdir -p '{save_path}'
!cp '{model_path}' '{save_path}/{new_file_name}'

In [26]:
!python /content/drive/MyDrive/StableDifussion/sd-scripts/gen_img_diffusers.py \
  --ckpt /content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt \
  --n_iter 1 \
  --scale 7.5 \
  --steps 20 \
  --outdir /content  \
  --prompt "a boy,((((cb_satoo))))" \
  --xformers \
  --W 512 \
  --H 512 \
  --seed   1 \
  --sampler k_euler_a \
  --network_module networks.lora \
  --network_weights /content/drive/MyDrive/my_models/canbright_satoo/satoo_with_descriptions_10000.safetensors \
  --network_mul 1 \
  --max_embeddings_multiples 3 \
  --clip_skip 1 \
  --batch_size 1 \
  --images_per_prompt 1 
  #--bf16 \

2023-06-13 10:21:09.987688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 10:21:10.192986: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-13 10:21:10.983118: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-06-13 10:21:10.983201: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cann

In [27]:
# *for test: remove "caption" and "tags" 

import json

with open('/content/drive/MyDrive/StableDifussion/sd-scripts/meta_lat.json') as f:
    data = json.load(f)

for key in data:
    if "caption" in data[key]:
        del data[key]["caption"]
    if "tags" in data[key]:
        del data[key]["tags"]

with open('/content/drive/MyDrive/StableDifussion/sd-scripts/modified_meta_lat.json', 'w') as f:
    json.dump(data, f)


In [ ]:
# write dataset config
import toml
instance_data_dir='/content/drive/MyDrive/datasets/imgs/with_txt_resized_canbright_imgs_1_partial/'

dataset_config={
    'general': {
        'enable_bucket': True
    },
    'datasets': [
        {
            'resolution': 512,
            'batch_size': 4,
            'subsets': [
                {
                    'image_dir': instance_data_dir,
                    'caption_extension': '.txt',
                    'num_repeats': 1
                }
            ]
        }
    ]
}
dataset_config_path='/content/drive/MyDrive/ML_Practices/Lora_test_1/lora/dataset_config.toml'
with open(dataset_config_path, 'w') as file:
    toml.dump(dataset_config, file)